### SPARKSQL Project

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.json("census_2010.json")
df.createOrReplaceTempView("census2010")
tables_result = spark.sql("SHOW TABLES")
tables_result.show()
result = spark.sql("select total ,year from census2010")
result.show(10)

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|         |census2010|       true|
+---------+----------+-----------+

+-------+----+
|  total|year|
+-------+----+
|4079669|2010|
|4085341|2010|
|4089295|2010|
|4092221|2010|
|4094802|2010|
|4097728|2010|
|4101686|2010|
|4107361|2010|
|4115441|2010|
|4126617|2010|
+-------+----+
only showing top 10 rows


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.json("census_2010.json")
df.createOrReplaceTempView("census2010")
result = spark.sql("""SELECT age,females,males from census2010 WHERE
                   age BETWEEN 0 AND 9 OR age >= 91
                   """)
result.show()

+---+-------+-------+
|age|females|  males|
+---+-------+-------+
|  0|1994141|2085528|
|  1|1997991|2087350|
|  2|2000746|2088549|
|  3|2002756|2089465|
|  4|2004366|2090436|
|  5|2005925|2091803|
|  6|2007781|2093905|
|  7|2010281|2097080|
|  8|2013771|2101670|
|  9|2018603|2108014|
| 91| 249925| 107133|
| 92| 204661|  80196|
| 93| 166201|  60163|
| 94| 133397|  45473|
| 95| 102645|  32664|
| 96|  74056|  22021|
| 97|  51337|  13994|
| 98|  35778|   8321|
| 99|  25673|   4612|
|100|  51007|   9506|
+---+-------+-------+



In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.json("census_2010.json")
df.createOrReplaceTempView("census2010")
result = spark.sql("""
select age,females,males,ROUND(females / males,4) AS female_male_ratio
FROM census2010
""")
result.show(5)

+---+-------+-------+-----------------+
|age|females|  males|female_male_ratio|
+---+-------+-------+-----------------+
|  0|1994141|2085528|           0.9562|
|  1|1997991|2087350|           0.9572|
|  2|2000746|2088549|            0.958|
|  3|2002756|2089465|           0.9585|
|  4|2004366|2090436|           0.9588|
+---+-------+-------+-----------------+
only showing top 5 rows


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.json("census_2010.json")
df.createOrReplaceTempView("census2010")
result = spark.sql("""
select age,total,females,ROUND(females / males,3) AS female_male_ratio
FROM census2010
ORDER BY female_male_ratio
""")
result.show()

+---+-------+-------+-----------------+
|age|  total|females|female_male_ratio|
+---+-------+-------+-----------------+
| 16|4410804|2146942|            0.948|
| 17|4451147|2165852|            0.948|
| 18|4454165|2168175|            0.948|
| 15|4347028|2117689|             0.95|
| 19|4432260|2159571|             0.95|
| 14|4285424|2089651|            0.952|
| 20|4411138|2151448|            0.952|
| 13|4220043|2060100|            0.954|
| 21|4384965|2140926|            0.954|
|  0|4079669|1994141|            0.956|
| 12|4169316|2037286|            0.956|
|  1|4085341|1997991|            0.957|
| 10|4137506|2023289|            0.957|
| 11|4144742|2026352|            0.957|
| 22|4362678|2133510|            0.957|
|  2|4089295|2000746|            0.958|
|  8|4115441|2013771|            0.958|
|  9|4126617|2018603|            0.958|
|  3|4092221|2002756|            0.959|
|  4|4094802|2004366|            0.959|
+---+-------+-------+-----------------+
only showing top 20 rows


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.json("census_2010.json")
df.createOrReplaceTempView("census2010")
result = spark.sql("""
select MAX(total) as max_population,MIN(total) as min_population
,COUNT(*) AS total_age_groups FROM census2010
""")
result.show(5)

+--------------+--------------+----------------+
|max_population|min_population|total_age_groups|
+--------------+--------------+----------------+
|       4596159|         30285|             101|
+--------------+--------------+----------------+



In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df_1980 = spark.read.json('census_1980.json')
df_1990 = spark.read.json('census_1990.json')
df_2000 = spark.read.json('census_2000.json')
df_2010 = spark.read.json('census_2010.json')
df_1980.createOrReplaceTempView("census1980")
df_1990.createOrReplaceTempView("census1990")
df_2000.createOrReplaceTempView("census2000")
df_2010.createOrReplaceTempView("census2010")
combined_census = spark.sql("""
select * from census1980
UNION ALL 
SELECT * FROM census1990
UNION ALL 
SELECT * FROM census2000 
UNION ALL 
SELECT * from census2010
""")
combined_census.createOrReplaceTempView("census_all_decades")
result = spark.sql("""
SELECT year,age,total from census_all_decades 
where age = 0 ORDER BY year
""")
result.show()
                                

+----+---+-------+
|year|age|  total|
+----+---+-------+
|1980|  0|3438584|
|1990|  0|3857376|
|2000|  0|3733034|
|2010|  0|4079669|
+----+---+-------+



In [7]:
# The four decades of census data have been merged into a temporary view called "census_all_decades"
result = spark.sql("""
select year,SUM(females) AS total_females,AVG(males) as avg_males
FROM census_all_decades
GROUP BY year
ORDER BY year 
""")
result.show()

+----+-------------+------------------+
|year|total_females|         avg_males|
+----+-------------+------------------+
|1980|    118348214|1107209.3762376239|
|1990|    130460736|1228177.3366336634|
|2000|    144922525|1382889.8019801981|
|2010|    158717489|1520095.3168316833|
+----+-------------+------------------+



In [8]:

result = spark.sql("""
select year,SUM(females) as total_females
,SUM(males) as total_males
,ROUND(SUM(females) / SUM(males), 4) as female_male_ratio
FROM census_all_decades 
where males > females
GROUP BY year
ORDER BY year
""")
result.show()

+----+-------------+-----------+-----------------+
|year|total_females|total_males|female_male_ratio|
+----+-------------+-----------+-----------------+
|1980|     42380049|   43938737|           0.9645|
|1990|     53915217|   56079874|           0.9614|
|2000|     71315207|   74166352|           0.9616|
|2010|     70512824|   73152939|           0.9639|
+----+-------------+-----------+-----------------+



In [9]:
result_SPARK_SQL = spark.sql("""SELECT year, 
       CASE WHEN age < 18 THEN 'Youth' ELSE 'Adult' END AS category,
       AVG(total) as avg_population
FROM census_all_decades
WHERE total > 50000
GROUP BY year, CASE WHEN age < 18 THEN 'Youth' ELSE 'Adult' END
HAVING AVG(total) > 100000
ORDER BY year, category
""")
result.show()

+----+-------------+-----------+-----------------+
|year|total_females|total_males|female_male_ratio|
+----+-------------+-----------+-----------------+
|1980|     42380049|   43938737|           0.9645|
|1990|     53915217|   56079874|           0.9614|
|2000|     71315207|   74166352|           0.9616|
|2010|     70512824|   73152939|           0.9639|
+----+-------------+-----------+-----------------+

